<a href="https://colab.research.google.com/github/TaraRasti/Deep_Learning_F20_Assignments/blob/master/Assignment4%5Csequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [2]:
pathdata='/content/drive/My Drive/dataFMNIST'

transformations = transforms.Compose([#transforms.Resize((32,32)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([.5],[.5])
                                      ])


data=datasets.FashionMNIST(pathdata,train=True, download=True,transform=transformations)

train_set, test_set = torch.utils.data.random_split(data, [42000, 18000])

train_loader = torch.utils.data.DataLoader(train_set,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,shuffle=True)

In [3]:

model = nn.Sequential(nn.Linear(784, 512),
                      nn.Sigmoid(),
                      nn.Linear(512, 256),
                      nn.ReLU(),
                      nn.Linear(256,128),
                      nn.ReLU(),
                      nn.Linear(128,64),
                      nn.ReLU(),
                      nn.Linear(64,32),
                      nn.ReLU(),
                      nn.Linear(32, 10),
                      nn.LogSoftmax(dim=1))

In [4]:
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=64, bias=True)
  (7): ReLU()
  (8): Linear(in_features=64, out_features=32, bias=True)
  (9): ReLU()
  (10): Linear(in_features=32, out_features=10, bias=True)
  (11): LogSoftmax(dim=1)
)


In [5]:
device = torch.device("cuda:0")
model = model.to(device)

In [6]:


epochs = 10
num_classes = 10
batch_size = 100
lr = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# Train the model
total_step = len(train_loader)
for epoch in range(epochs):
    for i, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        x = x.view(x.shape[0], -1)
        outputs = model(x)
        loss = criterion(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/42000], Loss: 2.2824
Epoch [1/10], Step [200/42000], Loss: 2.1037
Epoch [1/10], Step [300/42000], Loss: 2.4121
Epoch [1/10], Step [400/42000], Loss: 1.8854
Epoch [1/10], Step [500/42000], Loss: 2.0454
Epoch [1/10], Step [600/42000], Loss: 0.8005
Epoch [1/10], Step [700/42000], Loss: 2.8130
Epoch [1/10], Step [800/42000], Loss: 1.9708
Epoch [1/10], Step [900/42000], Loss: 4.3596
Epoch [1/10], Step [1000/42000], Loss: 1.0237
Epoch [1/10], Step [1100/42000], Loss: 2.1008
Epoch [1/10], Step [1200/42000], Loss: 1.3222
Epoch [1/10], Step [1300/42000], Loss: 0.8788
Epoch [1/10], Step [1400/42000], Loss: 1.5732
Epoch [1/10], Step [1500/42000], Loss: 1.0279
Epoch [1/10], Step [1600/42000], Loss: 0.3563
Epoch [1/10], Step [1700/42000], Loss: 1.0809
Epoch [1/10], Step [1800/42000], Loss: 0.0280
Epoch [1/10], Step [1900/42000], Loss: 1.4372
Epoch [1/10], Step [2000/42000], Loss: 0.9200
Epoch [1/10], Step [2100/42000], Loss: 1.4383
Epoch [1/10], Step [2200/42000], Loss: 1.28

In [7]:
model.eval() 
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        x = x.view(x.shape[0], -1)
        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    print('Test Accuracy: {} %'.format(100 * correct / total))

torch.save(model.state_dict(), 'model.checkpoint')

Test Accuracy: 80.02777777777777 %
